In [300]:
import pandas as pd
import numpy as np

In [301]:
plays_df = pd.read_csv("./data2023/plays.csv")
plays_df = plays_df.loc[plays_df['passResult'].isin(['C', 'I', 'IN'])]
player_stats_df = pd.read_csv("merged_df.csv").drop('Unnamed: 0', axis='columns')
backup_player_stats_df = pd.read_csv('data2023/players.csv')
medians_df = pd.read_csv('all_pos_df.csv').rename(columns={'Unnamed: 0': 'Position'})

In [302]:
plays_df['passResult'].unique()

array(['I', 'C', 'IN'], dtype=object)

In [303]:
player_stats_df["abrev_name"] = player_stats_df.displayName.apply(lambda x: str(x)[0]+ "."+"".join(str(x).split(" ")[1:]))
player_stats_df['abrev_name']

0                T.Brady
1       B.Roethlisberger
2              A.Rodgers
3          R.Fitzpatrick
4                M.Lewis
              ...       
1409         J.Jefferson
1410             D.Milne
1411            G.Stuard
1412            J.Jacobs
1413            J.Heflin
Name: abrev_name, Length: 1414, dtype: object

In [304]:
backup_player_stats_df["abrev_name"] = backup_player_stats_df.displayName.apply(lambda x: str(x)[0]+ "."+"".join(str(x).split(" ")[1:]))
backup_player_stats_df['abrev_name']

0                T.Brady
1       B.Roethlisberger
2               J.Peters
3              A.Rodgers
4          R.Fitzpatrick
              ...       
1674           F.Merrill
1675          R.McCollum
1676            J.Heflin
1677            J.Curhan
1678            F.Franks
Name: abrev_name, Length: 1679, dtype: object

In [305]:
# Remove time from play description
playDescText = plays_df.playDescription.apply(lambda x: " ".join(str(x).split()[1:]))
# Remove formation from play description
playDescText = playDescText.apply(lambda x: "".join(x.split(")")[1:]) if str(x).startswith("(") else x)
playDescText

0         T.Brady pass incomplete deep right to C.Godwin.
1        D.Prescott pass deep left to A.Cooper pushed ...
2        D.Prescott pass short middle to D.Schultz to ...
3         D.Prescott pass incomplete deep left to C.Lamb.
4        D.Prescott pass incomplete short left to C.La...
                              ...                        
8549     P.Mahomes pass incomplete short right [O.Ximi...
8550     P.Mahomes pass short right intended for B.Pri...
8551     P.Mahomes pass short right to T.Kelce to KC 3...
8553     D.Jones pass short right to E.Engram pushed o...
8555     D.Jones pass incomplete short right to E.Engram.
Name: playDescription, Length: 7565, dtype: object

In [306]:
# Create columns of names of players who carried out play
# People who pass the ball
passers = playDescText.map(lambda x: str(x).lower().split(" ")[str(x).lower().split(" ").index("pass")-1] if "pass" in str(x).lower().split(" ") else np.nan)

# People who received the ball
receivers = playDescText.map(lambda x: str(x).lower().split(" ")[str(x).lower().split(" ").index("to")+1] if ("pass" in str(x).lower().split(" ") and "to" in str(x).lower().split(" ")) & ("intercept" not in str(x).lower()) else np.nan)
# Remove full stop from some names at end
receivers.loc[receivers.map(lambda x: str(x)[-1]) == "."] = receivers.loc[receivers.map(lambda x: str(x)[-1]) == "."].apply(lambda x: "".join(str(x)[:-1]))
# 2 values have NO # instead of names, so we can remove these
receivers.loc[receivers=="no"] = np.nan
receivers.loc[~receivers.str.contains('.', na=False, regex=False)] = np.nan

In [307]:
key_players_df = pd.DataFrame({'gameId': plays_df['gameId'], 'playId': plays_df['playId'], 'passer': passers, 'receiver': receivers})
key_players_df = key_players_df.dropna()
bad_players = np.concatenate((key_players_df[key_players_df['passer'].str.index('.') != 1]['passer'].unique(),
                                     key_players_df[key_players_df['receiver'].str.index('.') != 1]['receiver'].unique(),
                                     ['a.st']))
print(bad_players)
key_players_df = key_players_df.drop(key_players_df[(key_players_df['passer'].isin(bad_players)) | (key_players_df['receiver'].isin(bad_players))].index)
key_players_df = key_players_df.reset_index(drop=True)
key_players_df

['ty.taylor' 'aa.rodgers' 'jos.allen' 'dj.moore' 'mi.carter' 'am.rodgers'
 'da.williams' 'a.st']


,gameId,playId,passer,receiver
0,2021090900,97,t.brady,c.godwin
1,2021090900,137,d.prescott,a.cooper
2,2021090900,187,d.prescott,d.schultz
3,2021090900,282,d.prescott,c.lamb
4,2021090900,349,d.prescott,c.lamb
...,...,...,...,...
6685,2021110100,3955,d.jones,d.booker
6686,2021110100,4016,p.mahomes,t.kelce
6687,2021110100,4113,p.mahomes,t.kelce
6688,2021110100,4363,d.jones,e.engram


In [308]:
def stats_from_name(name, stats, columns, backup_stats):
    player_stats = stats[stats['abrev_name'].str.lower() == name].drop(stats.columns.difference(columns), axis='columns')[:1]
    if player_stats.empty:
        position = backup_stats[backup_stats['abrev_name'].str.lower() == name]['officialPosition'].values[0]
        player_stats = medians_df[medians_df['Position'] == position][columns].sort_index(axis=1)
    return player_stats


def yards_from_play(gameId, playId, plays):
    return plays[(plays['gameId'] == gameId) & (plays['playId'] == playId)]['prePenaltyPlayResult'].values[0]

In [309]:
feature_columns = ['Age', 'Height', 'Weight', 'Overall', 'Speed',
 'Acceleration', 'Agility', 'Change of Dir', 'Strength', 'Jumping',
 'Awareness', 'Carrying', 'Break Tackle', 'Juke Move', 'Spin Move',
 'Trucking', 'Stiff Arm', 'BC Vision', 'Catching', 'Catch In Traffic',
 'Spec Catch', 'Release', 'Short RR', 'Medium RR', 'Deep RR',
 'Throw Power', 'Throw Acc Short', 'Throw Acc Mid', 'Throw Acc Deep',
 'Throw Under Pressure', 'Throw On The Run', 'Play Action', 'Break Sack',
 'Run Block', 'Run Block Power', 'Run Block Finesse', 'Pass Block',
 'Pass Block Power', 'Pass Block Finesse', 'Impact Blocking',
 'Lead Blocking', 'Tackle', 'Hit Power', 'Pursuit', 'Man Coverage',
 'Zone Coverage', 'Press', 'Play Recognition', 'Power Moves',
 'Finesse Moves', 'Block Shedding', 'Kick Power', 'Kick Accuracy',
 'Kick Return', 'Stamina', 'Injury', 'Toughness', 'Years Pro']

In [310]:
X = np.stack(key_players_df.apply(lambda x: np.concatenate((stats_from_name(x['passer'], player_stats_df, feature_columns, backup_player_stats_df),
                                                            stats_from_name(x['receiver'], player_stats_df, feature_columns, backup_player_stats_df)), axis=1)[0], axis=1)).astype(float)
X.shape


(6690, 116)

In [311]:
Y = key_players_df.apply(lambda x: yards_from_play(x['gameId'], x['playId'], plays_df), axis=1).to_numpy().astype(float)
Y.shape

(6690,)

In [312]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, random_state=0)

In [313]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(X_train, Y_train)
reg.score(X_test, Y_test)

-1.4217083489019843e+22

In [314]:
from sklearn.linear_model import ElasticNet
reg = ElasticNet().fit(X_train, Y_train)
reg.score(X_test, Y_test)

0.006389312175957618

In [315]:
from sklearn import svm
reg = svm.SVR().fit(X_train, Y_train)
reg.score(X_test, Y_test)

-0.06954036645002493

In [316]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
k_best = SelectKBest(mutual_info_regression).fit(X_train, Y_train)
Xk_train = k_best.transform(X_train)
Xk_test = k_best.transform(X_test)
k_best.get_feature_names_out()

array(['x27', 'x66', 'x68', 'x70', 'x83', 'x87', 'x94', 'x97', 'x98',
       'x115'], dtype=object)

In [317]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(Xk_train, Y_train)
reg.score(Xk_test, Y_test)

0.005782448623484759

In [318]:
from sklearn.linear_model import ElasticNet
reg = ElasticNet().fit(Xk_train, Y_train)
reg.score(Xk_test, Y_test)

0.006566129431764822

In [319]:
from sklearn import svm
reg = svm.SVR().fit(Xk_train, Y_train)
reg.score(Xk_test, Y_test)

-0.06795613446533055

In [320]:
from sklearn import tree
reg = tree.DecisionTreeRegressor().fit(X_train, Y_train)
reg.score(X_test, Y_test)

-0.029549048047659676

In [321]:
from sklearn.linear_model import SGDRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
reg = make_pipeline(StandardScaler(), SGDRegressor()).fit(X_train, Y_train)
reg.score(X_test, Y_test)

-0.029799642284264127

In [322]:
from sklearn.neural_network import MLPRegressor
reg = MLPRegressor().fit(X_train, Y_train)
reg.score(X_test, Y_test)

-0.017374317740058487